In [1]:
# import sys
# sys.path.append('/home/sn/ERAv1/main_repo')

In [110]:
import torch, torchvision
from torchvision import transforms
import numpy as np
import gradio as gr
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import random
import config
import cv2
from utils import non_max_suppression, cells_to_bboxes
from PIL import Image

# from models.resnet import ResNet18
from model import s13Model, ScalePrediction, CNNBlock, ResidualBlock

import gradio as gr

In [ ]:
# device_id=1
# torch.cuda.set_device(device_id)

In [4]:
# model = s13Model(num_classes=config.NUM_CLASSES)

# new_model = model.load_from_checkpoint('40th_epoch_s13Model.ckpt')

RuntimeError: Error(s) in loading state_dict for s13Model:
	size mismatch for layers.15.pred.1.conv.weight: copying a param with shape torch.Size([75, 1024, 1, 1]) from checkpoint, the shape in current model is torch.Size([255, 1024, 1, 1]).
	size mismatch for layers.15.pred.1.conv.bias: copying a param with shape torch.Size([75]) from checkpoint, the shape in current model is torch.Size([255]).
	size mismatch for layers.15.pred.1.bn.weight: copying a param with shape torch.Size([75]) from checkpoint, the shape in current model is torch.Size([255]).
	size mismatch for layers.15.pred.1.bn.bias: copying a param with shape torch.Size([75]) from checkpoint, the shape in current model is torch.Size([255]).
	size mismatch for layers.15.pred.1.bn.running_mean: copying a param with shape torch.Size([75]) from checkpoint, the shape in current model is torch.Size([255]).
	size mismatch for layers.15.pred.1.bn.running_var: copying a param with shape torch.Size([75]) from checkpoint, the shape in 

In [ ]:
# torch.save(new_model.state_dict(), 's13Model.pth')

In [5]:
# # model = ResNet18()
model = s13Model(num_classes=config.NUM_CLASSES)

# # new_model = model.load_from_checkpoint('s10Model.ckpt')
model.load_state_dict(torch.load("s13Model.pth", map_location=torch.device('cpu')), strict=False)
model.eval()

<All keys matched successfully>

In [140]:
# import torch, torchvision
# from torchvision import transforms
# import numpy as np
# import gradio as gr
# from PIL import Image
# from pytorch_grad_cam import GradCAM
# from pytorch_grad_cam.utils.image import show_cam_on_image
# from resnet import ResNet18
# import gradio as gr

# model = ResNet18()
# model.load_state_dict(torch.load("model.pth", map_location=torch.device('cpu')), strict=False)

# inv_normalize = transforms.Normalize(
#     mean=[-0.50/0.23, -0.50/0.23, -0.50/0.23],
#     std=[1/0.23, 1/0.23, 1/0.23]
# )
inv_normalize = transforms.Normalize(
    mean=[-0, -0, -0],
    std=[1, 1, 1]
)

classes = [
    "aeroplane",
    "bicycle",
    "bird",
    "boat",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "diningtable",
    "dog",
    "horse",
    "motorbike",
    "person",
    "pottedplant",
    "sheep",
    "sofa",
    "train",
    "tvmonitor"
]

# This will help us create a different color for each class
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))
scaled_anchors = (torch.tensor(config.ANCHORS)
    * torch.tensor(config.S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)).to('cpu')


def draw_boxes(image, boxes):
# Create a Rectangle patch
    
    image = np.array(image)
#     image = np.clip(image, a_min = -0.999999, a_max = 0.999999)
#     print(image.max())
    height, width, _ = image.shape
    for box in boxes:
#         print(image)
        assert len(box) == 6, "box should contain class pred, confidence, x, y, width, height"
        class_pred = int(box[0])
        box = box[2:]
#         print('box is ', box)
        upper_left_x = box[0] - box[2] / 2
        upper_left_y = box[1] - box[3] / 2
        lower_right_x = box[0] + box[2] / 2
        lower_right_y = box[1] + box[3] / 2
#         print((upper_left_x, upper_left_y), (lower_right_x, lower_right_y))
        color = COLORS[class_pred]
        cv2.rectangle(
            image,
            (int(upper_left_x*width), int(upper_left_y*height)),
            (int(lower_right_x*width), int(lower_right_y*height)),
            color, 2
        )
        cv2.putText(image, classes[class_pred], (int(upper_left_x*width), int(upper_left_y*height - 5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2,
                    lineType=cv2.LINE_AA)
    return image

def get_bbox_image(images, out, thresh=0.6, iou_thresh=0.5, anchors=scaled_anchors):
#     model.eval()
#     x, y = next(iter(loader))
#     x = x.to("cuda")
#     with torch.no_grad():
#         out = model(x)
#     i = 0
    x = images
    bboxes = [[] for _ in range(x.shape[0])]
    for i in range(3):
        batch_size, A, S, _, _ = out[i].shape
        anchor = anchors[i]
        boxes_scale_i = cells_to_bboxes(
            out[i], anchor, S=S, is_preds=True
        )
        for idx, (box) in enumerate(boxes_scale_i):
            bboxes[idx] += box
#     print('bbox length ', len(bboxes[0]))
    nms_boxes = non_max_suppression(
        bboxes[0], iou_threshold=iou_thresh, threshold=thresh, box_format="midpoint",
    )
    return draw_boxes(x, nms_boxes)
#     return draw_boxes(x.permute(1,2,0).detach().cpu(), nms_boxes)


def inference(input_img, transparency = 0.5, target_layer_number = -1):
    org_img = input_img
    input_img_aug = config.test_transforms(image=input_img, bboxes=[])
    input_img = input_img_aug["image"]
    input_img = input_img
    input_img = input_img.unsqueeze(0)
    outputs = model(input_img)
#     print('outputs ', outputs)
    obj_detected = get_bbox_image(org_img, outputs)
#     softmax = torch.nn.Softmax(dim=0)
#     o = softmax(outputs.flatten())
#     confidences = {classes[i]: float(o[i]) for i in range(10)}
#     _, prediction = torch.max(outputs, 1)
#     target_layers = [model.layers[target_layer_number]]
#     cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)
#     grayscale_cam = cam(input_tensor=input_img, targets=[ClassifierOutputTarget(1)])
#     grayscale_cam = grayscale_cam[0, :]
#     img = input_img.squeeze(0)
#     img = inv_normalize(img)
#     rgb_img = np.transpose(img, (1, 2, 0))
#     rgb_img = rgb_img.numpy()
#     visualization = show_cam_on_image(org_img/255, grayscale_cam, use_rgb=True, image_weight=transparency)
    return obj_detected/255 #, visualization

title = "PASCAL VOC 2007 Dataset trained on Custom Model with GradCAM"
description = "A simple Gradio interface for Object Detection using Custom model, and get GradCAM"
examples = [["example_imgs/cat.jpeg"], 
            ["example_imgs/dog.jpeg"],
            ['example_imgs/dogs.jpeg'],
            ['example_imgs/train.jpeg'],
            ['example_imgs/bird1.jpeg'],
            ['example_imgs/cars1.jpeg'],
            ['example_imgs/horse1.jpeg'],
            ['example_imgs/train2.jpeg'],
            ['example_imgs/bird2.jpeg'],
            ['example_imgs/cars2.jpeg'],
            ['example_imgs/horse2.jpeg'],

           ]
demo = gr.Interface(
    inference, 
    inputs = [gr.Image(shape=(416, 416), label="Input Image")], 
#               gr.Slider(0, 1, value = 0.5, label="Opacity of GradCAM"), 
#               gr.Slider(-2, -1, value = -2, step=1, label="Which Layer?")], 
#     outputs = [ #gr.Label(num_top_classes=5), 
    outputs = [gr.Image(shape=(416, 416), label="Output").style(width=256, height=256)],
    title = title,
    description = description,
    examples = examples,
)
demo.launch()

/tmp/ipykernel_6226/701503499.py:157: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  outputs = [gr.Image(shape=(416, 416), label="Output").style(width=256, height=256)],


Running on local URL:  http://127.0.0.1:7916

To create a public link, set `share=True` in `launch()`.


In [5]:

# # # model = ResNet18()
# # model = s10Model()

# # new_model = model.load_from_checkpoint('s10Model.ckpt')
# # # model.load_state_dict(torch.load("s10Model.pth", map_location=torch.device('cpu')), strict=False)

# inv_normalize = transforms.Normalize(
# mean=[-0.50/0.23, -0.50/0.23, -0.50/0.23],
# std=[1/0.23, 1/0.23, 1/0.23]
# )

# new_model = model

# classes = config.PASCAL_CLASSES
# # ('plane', 'car', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck')

# image_fname_prefix=['cat', 'dog', 'bird1', 'bird2', 'bird3','cars1', 'cars2', 'horse1', 'horse2', 'boat1']

# target = ['cat', 'dog', 'bird', 'bird', 'bird','car', 'car', 'horse', 'horse', 'ship']

# def display_gradcam(num_images=1, top_classes=3, transparency=0.5,target_layer_number=-1, max_images=5):
    
#     #Set up gradcam instance
#     target_layers = [model.layers.28[target_layer_number]]
#     cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)
#     transform = transforms.ToTensor()
    
#     confidences_list = []
#     visualization_list = []
    
#     for i in range(num_images):
# #         a = random.randint(0,max_images-1)
#         f_prefix = image_fname_prefix[i]
# #         print(f'example_imgs/{f_prefix}.jpeg')
#         input_image = Image.open(f'example_imgs/{f_prefix}.jpeg').resize((32,32))
#         org_img = np.array(input_image).astype(np.uint8)
#         input_img = transform(input_image)
#         input_img = input_img
#         input_img = input_img.unsqueeze(0)
#         print(input_img.shape)

#         outputs = model(input_img)
#         softmax = torch.nn.Softmax(dim=0)

#         o = softmax(outputs.flatten())
#     #     o = outputs.flatten()

#         confidences = {classes[i]: float(o[i]) for i in range(10)}

#         _, prediction = torch.max(outputs, 1)

#         grayscale_cam = cam(input_tensor=input_img, targets=None)
#         grayscale_cam = grayscale_cam[0, :]

#         img = input_img.squeeze(0)
#         img = inv_normalize(img)
#         rgb_img = np.transpose(img, (1, 2, 0))
#         rgb_img = rgb_img.numpy()
#         visualization = show_cam_on_image(org_img/255, grayscale_cam, use_rgb=True, image_weight=transparency)
#         confidences_list.append(confidences)
#         visualization_list.append((visualization, classes[prediction]))
# #         caption_list.append(f_prefix)

#     return visualization_list, confidences_list
        
# def display_misClassified(num_images=1, max_images=10):

#     transform = transforms.ToTensor()
    
#     confidences_list = []
#     visualization_list = []
    
#     for i in range(max_images):
#         f_prefix = image_fname_prefix[i]
# #         print(f'example_imgs/{f_prefix}.jpeg')
#         input_image = Image.open(f'example_imgs/{f_prefix}.jpeg').resize((32,32))
#         org_img = np.array(input_image).astype(np.uint8)
#         input_img = transform(input_image)
#         input_img = input_img
#         input_img = input_img.unsqueeze(0)
# #         print(input_img.shape)

#         outputs = model(input_img)
#         softmax = torch.nn.Softmax(dim=0)

#         o = softmax(outputs.flatten())
#     #     o = outputs.flatten()

#         confidences = {classes[i]: float(o[i]) for i in range(10)}

#         _, prediction = torch.max(outputs, 1)
        
#         if classes[prediction] != target[i]:
#             confidences_list.append(confidences)
#             visualization_list.append((org_img/255, f'{classes[prediction]} / {target[i]}'))

#     return visualization_list, confidences_list

# def inference(input_img, img_type='GradCAM', num_images = 1, top_classes=3,transparency = 0.5, 
#               target_layer_number = -1):

#     transform = transforms.ToTensor()
#     org_img = input_img
#     print(org_img.dtype)
#     input_img = transform(input_img)
    
#     if img_type == 'GradCAM':
#         v_list, c_list = display_gradcam(num_images, top_classes, transparency,target_layer_number, max_images=10)
#     elif img_type == 'Mis-Classified':
#         v_list, c_list = display_misClassified(num_images)
#     else:
#         target_layers = [model.resblock2[target_layer_number]]
#         cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)
#         input_img = input_img
#         input_img = input_img.unsqueeze(0)

#         outputs = new_model(input_img)
#         softmax = torch.nn.Softmax(dim=0)

#         o = softmax(outputs.flatten())

#         confidences = {classes[i]: float(o[i]) for i in range(10)}

#         _, prediction = torch.max(outputs, 1)

#         grayscale_cam = cam(input_tensor=input_img, targets=None)
#         grayscale_cam = grayscale_cam[0, :]

#         img = input_img.squeeze(0)
#         img = inv_normalize(img)
#         rgb_img = np.transpose(img, (1, 2, 0))
#         rgb_img = rgb_img.numpy()
#         visualization = show_cam_on_image(org_img/255, grayscale_cam, use_rgb=True, image_weight=transparency)
#         v_list = [(visualization, classes[prediction])]
#     return  v_list #c_list

# #     return top_classes, confidences_list, visualization_list

# title = "PASCAL VOC 2007 Dataset trained on Custom Model with GradCAM"
# description = "A simple Gradio interface for Object Detection using Custom model, and get GradCAM"
# examples = [["example_imgs/cat.jpeg",'GradCAM',5,5, 0.5, -1], 
#             ["example_imgs/dog.jpeg",'GradCAM',5,5, 0.5, -1],
#             ["example_imgs/bird1.jpeg",'GradCAM',5,5, 0.5, -1],
#             ["example_imgs/bird2.jpeg",'GradCAM',5,5, 0.5, -1],
#             ["example_imgs/bird3.jpeg",'GradCAM',5,5, 0.5, -1],
#             ["example_imgs/cars1.jpeg",'GradCAM',5,5, 0.5, -1],
#             ["example_imgs/cars2.jpeg",'GradCAM',5,5, 0.5, -1],
#             ["example_imgs/horse1.jpeg",'GradCAM',5,5, 0.5, -1],
#             ["example_imgs/horse2.jpeg",'GradCAM',5,5, 0.5, -1],
#            ]

# demo = gr.Interface(
#     inference, 
#     inputs = [gr.Image(shape=416, 416), label="Input Image"), 
# #               gr.Radio(['GradCAM', 'Mis-Classified'],label="Type of Images", info="Do you want to see GradCam or Mis-Classified images?"), 
# #               gr.Slider(1, 10, value=1,step=1, label="How Many?", info="Choose between 1 and 5"),
#               gr.Slider(1, 10, value=1,step=1, label="How Many top-classes?", info="Choose between 1 and 10"),
#               gr.Slider(0, 1, value = 0.5, label="Opacity of GradCAM"), 
#               gr.Slider(-2, -1, value = -2, step=1, label="Which Layer for GradCAM?")], 
#     outputs = [gr.Gallery(
#             label="Generated images", show_label=False, elem_id="gallery"
#         ).style(columns=[3], rows=[2], object_fit="contain", height="auto")
#               ],
#     title = title,
#     description = description,
#     examples = examples,
# )
# demo.launch()

/tmp/ipykernel_15110/2082156291.py:161: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  outputs = [gr.Gallery(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


uint8
uint8
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
uint8
